In [77]:
#from google.colab import drive
import requests
import re
import os
DRIVE_PATH = "/content/drive/MyDrive/LLMProject/"
FILENAME = "scifi_llm_dataset.txt"
#OUTPUT_FILE = os.path.join(DRIVE_PATH, FILENAME)
OUTPUT_FILE  = "/kaggle/input/modeldata/scifi_llm_dataset.txt"
#drive.mount('/content/drive')

In [78]:

#Desired Sci-Fi authors for training the llm

BOOK_IDS = {
     # H.G. Wells
    '35': 'The Time Machine by H. G. Wells',
    '36': 'The War of the Worlds by H. G. Wells',
    '5230': 'The Invisible Man by H. G. Wells',
    '159': 'The Island of Doctor Moreau by H. G. Wells',
    '1013': 'The First Men in the Moon by H. G. Wells',

    # Jules Verne
    '32': 'Journey to the Center of the Earth by Jules Verne',
    '105': 'From the Earth to the Moon by Jules Verne',
    '164': 'Twenty Thousand Leagues under the Sea by Jules Verne',
    '1268': 'The Mysterious Island by Jules Verne',
    '83': 'Around the Moon by Jules Verne',

    # Edgar Rice Burroughs
    '64317': 'A Princess of Mars by Edgar Rice Burroughs',
    '68': 'The Gods of Mars by Edgar Rice Burroughs',

    # Arthur Conan Doyle
    '1952': 'The Lost World by Arthur Conan Doyle',
    '22357': 'The Poison Belt by Arthur Conan Doyle',

    # Mary Shelley & R.L. Stevenson (Sci-Fi Origins)
    '84': 'Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley',
    '16': 'The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson',

    # Other Influential Classics
    '97': 'Flatland: A Romance of Many Dimensions by Edwin Abbott Abbott',
    '21970': 'The Scarlet Plague by Jack London',
    '32032': 'Second Variety by Philip K. Dick',
    '1513': 'Moby Dick; Or, The Whale by Herman Melville', # Proto-sci-fi
    '73723': 'The Machine Stops by E. M. Forster',
    '19141': 'Edison\'s Conquest of Mars by Garrett Putnam Serviss',
    '32338': 'The Metal Monster by Abraham Merritt',
    '18452': 'The Hampdenshire Wonder by J. D. Beresford',
    '521': 'The Last Man by Mary Wollstonecraft Shelley'
}
def word_count(text):
  return len(re.findall(r'\b\w+\b', text.lower()))


In [79]:
collected_texts = []
total_words = 0
book_count = 0

print("Starting direct download of selected books...")
if os.path.exists(OUTPUT_FILE):
    print(f"Output file '{OUTPUT_FILE}' already exists. Skipping download.")
else:
  for book_id, title in BOOK_IDS.items():
      url = f"https://www.gutenberg.org/ebooks/{book_id}.txt.utf-8"

      try:
          # Download the book's text
          response = requests.get(url)
          response.raise_for_status()
          text = response.text

          #Clean the Gutenberg header and footer
          start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
          end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

          start_pos = text.find(start_marker)
          if start_pos != -1:
              start_pos = text.find("\n", start_pos) + 1
              text = text[start_pos:]

          end_pos = text.rfind(end_marker) # Use rfind to find the last occurrence
          if end_pos != -1:
              text = text[:end_pos]

          cleaned_text = text.strip()
          collected_texts.append(cleaned_text)

          current_words = word_count(cleaned_text)
          total_words += current_words
          book_count += 1
          print(f"Downloaded '{title}' | Words: {current_words:,}")

      except requests.exceptions.RequestException as e:
          print(f"Failed to download book ID {book_id}: {e}")

  print("\nFinished downloading all books.")
  if book_count > 0:
      print(f"Final word count: {total_words:,} from {book_count} books.")
      print(f"Saving all collected text to '{OUTPUT_FILE}'...")
      with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
          f.write("\n\n".join(collected_texts))
      print("Done")
  else:
      print("No books were downloaded.")

Starting direct download of selected books...
Output file '/kaggle/input/modeldata/scifi_llm_dataset.txt' already exists. Skipping download.


In [80]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    data = f.read()
data[:500]


'The Time Machine\n\nAn Invention\n\nby H. G. Wells\n\n\nCONTENTS\n\n I Introduction\n II The Machine\n III The Time Traveller Returns\n IV Time Travelling\n V In the Golden Age\n VI The Sunset of Mankind\n VII A Sudden Shock\n VIII Explanation\n IX The Morlocks\n X When Night Came\n XI The Palace of Green Porcelain\n XII In the Darkness\n XIII The Trap of the White Sphinx\n XIV The Further Vision\n XV The Time Traveller’s Return\n XVI After the Story\n Epilogue\n\n\n\n\n I.\n Introduction\n\n\nThe Time Traveller (for so it will '

In [81]:
#count the number of times each pair of encoded bytes appears together in the data
def get_stats(ids, counts = None):
  counts = {} if counts is None else counts
  for pair in zip(ids,ids[1:]):
    counts[pair] = counts.get(pair,0)+1
  return counts


In [82]:
#in the list of ids replace all consecutive pairs with the new token idx
def merge(ids,pair,idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i+=2
    else:
      newids.append(ids[i])
      i+=1
  return newids
print(merge([2,3,3,4,5,6],(3,4),333))

[2, 3, 333, 5, 6]


In [83]:
import regex as re
#Some hyperparameters for the training of the tokenizer
VOCAB_SIZE = 8256 # The number of tokens in our vocabulary
NUM_MERGES = VOCAB_SIZE - 256 # 8000 merges to obtain the desired size of the vocab
#I will use the split pattern from the gpt 4 as it is the most complete I could find
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""


In [84]:
#This function trains the tokenizer
def train(text:str):
  ids = []
  text_split = re.findall(GPT4_SPLIT_PATTERN, text)

  for chars in text_split:
    byte_list  = list(chars.encode("utf-8"))
    ids.append(byte_list)

  merges = {}
  vocab = {idx: bytes([idx]) for idx in range(256)}

  for i in range(NUM_MERGES):
    stats = {}
    for chars in ids:
      get_stats(chars,stats)
    if not stats:
      break
    pair = max(stats, key = stats.get)
    idx = 256 + i
    ids = [merge(chars, pair, idx) for chars in ids]
    merges[pair] = idx
    vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
  return vocab, merges


In [85]:
def encode(text:str, merges):
  tokens = list(text.encode("utf-8"))
  while len(tokens) >=2:
    stats = get_stats(tokens)
    pair = min(stats, key = lambda s: merges.get(s, float("inf")))
    if pair not in merges:
      break
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens
def decode(ids, vocab):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace") #replace inserts a special character for undecodable byte
  return text


vocab_dummy = {}
merges_dummy = {}
dummy_text = "This is a dummy text"
vocab_dummy, merges_dummy = train("This is a dummy mmm his is")
print(decode(encode(dummy_text, merges_dummy), vocab_dummy))

print(encode(dummy_text, merges_dummy))
print(list(dummy_text.encode("utf-8")))
print(vocab_dummy[265])

This is a dummy text
[260, 259, 261, 265, 32, 116, 101, 120, 116]
[84, 104, 105, 115, 32, 105, 115, 32, 97, 32, 100, 117, 109, 109, 121, 32, 116, 101, 120, 116]
b' dummy'


The base functions for the tokenizer are finished, now what is left is to create methods for loading and saving the tokens obtained during the training phase. After this I will start building the proper llm from scratch.

In [86]:
import unicodedata

#This function builds the vocabulary from the merges that we perform
def build_vocab(merges):
  vocab = {idx: bytes([idx]) for idx in range(256)}
  for (p0, p1), idx in merges.items():
      vocab[idx] = vocab[p0] + vocab[p1]
  return vocab

#We will also define two functions such that we can better visualize our merges and vocab dictionaries
def replace_control_chars(s: str):
    chars = []
    for ch in s:
        if unicodedata.category(ch)[0] != "C":
            chars.append(ch)
        else:
            chars.append(f"\\u{ord(ch):04x}") # escape
    return "".join(chars)

def render_token(t: bytes) -> str:
    # pretty print a token, escaping control characters
    s = t.decode('utf-8', errors='replace')
    s = replace_control_chars(s)
    return s

In [87]:
def save(vocab_file, model_file, merges, vocab):
    with open(model_file, 'w') as f:
            # the merges dict
            for idx1, idx2 in merges:
                f.write(f"{idx1} {idx2}\n")
    inverted_merges = {idx: pair for pair, idx in merges.items()} # this is needed to find the children of each token that was newly created
    with open(vocab_file, "w", encoding="utf-8") as f:
        for idx, token in vocab.items():
            s = render_token(token)
            # find the children of this token
            if idx in inverted_merges:
                # if this token has children show the merge
                idx0, idx1 = inverted_merges[idx]
                s0 = render_token(vocab[idx0])
                s1 = render_token(vocab[idx1])
                f.write(f"[{s0}][{s1}] -> [{s}] {idx}\n")
            else:
                f.write(f"[{s}] {idx}\n") # this case is for the first 256 tokens
def load(model_file):
    merges = {}
    idx = 256
    with open(model_file, 'r', encoding="utf-8") as f:
        # read the merges
        for line in f:
            idx1, idx2 = map(int, line.split())
            merges[(idx1, idx2)] = idx
            idx += 1
    vocab = build_vocab(merges)
    return vocab, merges

In [88]:
#In this cell I will test the tokenizer to see if it works correctly
MODEL_FILE = "/kaggle/input/modeldata/tokenizer.model"
VOCAB_FILE = "/kaggle/input/modeldata/tokenizer.vocab"
if os.path.exists(MODEL_FILE) and os.path.exists(VOCAB_FILE):
    print(f"Model file '{MODEL_FILE}' and vocab file '{VOCAB_FILE}' already exist. Skipping training.")
else:
    print("Training my tokenizer...(hope it works)")
    vocab, merges = train(data)
    print("Tokenizer trained (hopefully) ")
    print("Saving tokenizer...")
    save(VOCAB_FILE, MODEL_FILE, merges, vocab)
    print("Tokenizer saved")

print("Loading tokenizer")
loaded_vocab, loaded_merges = load(MODEL_FILE)
print("Tokenizer loaded")

print("Checking my tokenizer: ")
dummy_sentence = "I am confident my tokenizer works as it should"
print(f"Original sentence: {dummy_sentence}")

encoded_dummy = encode(dummy_sentence, loaded_merges)
print(f"Encoded sentence: {encoded_dummy}")

decoded_dummy = decode(encoded_dummy, loaded_vocab)
print(f"Decoded sentence: {decoded_dummy}")

assert dummy_sentence == decoded_dummy, "The tokenizer did not work correctly"
print("All is good, God bless")

Model file '/kaggle/input/modeldata/tokenizer.model' and vocab file '/kaggle/input/modeldata/tokenizer.vocab' already exist. Skipping training.
Loading tokenizer
Tokenizer loaded
Checking my tokenizer: 
Original sentence: I am confident my tokenizer works as it should
Encoded sentence: [73, 615, 1163, 1368, 358, 281, 1265, 780, 266, 4157, 347, 324, 677]
Decoded sentence: I am confident my tokenizer works as it should
All is good, God bless


In [89]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
torch.manual_seed(1337)

#Hyperparameters
block_size = 256
batch_size = 32
periodicity_of_evaluation = 100
learning_rate = 1e-3 #maybe change it depends on how the model performs
num_max_iterations = 10000
number_of_batches_to_eval = 200
dropout = 0.2
num_embeddings = 256
num_heads = 6
num_layers = 6



cuda


In [90]:
#In this cell I will save the new_data so I don't have to go again through the painstaking process of waiting for the whole dataset to be encoded
ENCODED_FILE = "/kaggle/input/modeldata/encoded_text"
if os.path.exists(ENCODED_FILE):
    print(f"Encoded file '{ENCODED_FILE}' already exist. Skipping encoding.")
    new_data = torch.load(ENCODED_FILE)
else:
  new_data = torch.tensor(encode(data,loaded_merges), dtype = torch.long)
  torch.save(new_data,ENCODED_FILE)


Encoded file '/kaggle/input/modeldata/encoded_text' already exist. Skipping encoding.


In [91]:
print(len(new_data))
# In this cell I will split the data but I will take chunks of size 20000 
# so my training data and validation data are from every book
# Previously I only took validation data from the last books so I believe my validation loss was measured inacuratelly
chunk_size = 20000
split_percentage = int(0.9*chunk_size)
train_chunks = []
validation_chunks = []

for i in range(0,len(new_data),chunk_size):
    current_chunk = new_data[i:(i+chunk_size)]
    train_chunks.append(current_chunk[:split_percentage])
    validation_chunks.append(current_chunk[split_percentage:])

train_data = torch.cat(train_chunks)
validation_data = torch.cat(validation_chunks)

print(f"Original data size: {len(new_data)}")
print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(validation_data)}")

2284374
Original data size: 2284374
Training data size: 2056374
Validation data size: 228000


In [92]:
def get_batch(split):
  inter_data = train_data if split == 'train' else validation_data
  ix = torch.randint(len(inter_data) - block_size, (batch_size,))
  x = torch.stack([inter_data[i:i+block_size] for i in ix])
  y = torch.stack([inter_data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x,y

@torch.no_grad()
def approximate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(number_of_batches_to_eval)
    for k in range(number_of_batches_to_eval):
      X,Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [93]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(num_embeddings, head_size, bias = False)
    self.query = nn.Linear(num_embeddings, head_size, bias = False)
    self.value = nn.Linear(num_embeddings, head_size, bias = False)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer('mask', torch.tril(torch.ones(block_size,block_size)))
  def forward(self, x): # x is the input
    num_batches, time_steps, channels = x.shape
    keys = self.key(x)
    queries = self.query(x)
    values = self.value(x)

    weights = queries @ keys.transpose(-2,-1) * channels**-0.5 # weights will be of size (B,T,T)
    weights = weights.masked_fill(self.mask[:time_steps,:time_steps] == 0, float('-inf'))
    weights = F.softmax(weights, dim = -1)
    weights = self.dropout(weights)
    output = weights @ values #output will be of size (B,T,C)
    return output

class MultiHead(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.projection = nn.Linear(num_heads * head_size, num_embeddings)
    self.dropout = nn.Dropout(dropout)
  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim = -1)
    out = self.dropout(self.projection(out))
    return out

class FeedForward(nn.Module):
  # This is a simple linear layer with a non-liner layer that follows it
  def __init__(self, num_embeddings):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(num_embeddings, 4*num_embeddings),
        nn.ReLU(),
        nn.Linear(4*num_embeddings, num_embeddings),
        nn.Dropout(dropout)
    )
  def forward(self, x):
    return self.net(x)


class TransformerBlock(nn.Module):
  # This is a whole transformer block
  def __init__(self, num_embeddings, num_heads):
    super().__init__()
    head_size = num_embeddings // num_heads
    self.multihead = MultiHead(num_heads, head_size)
    self.feedforward = FeedForward(num_embeddings)
    self.layernorm1 = nn.LayerNorm(num_embeddings)
    self.layernorm2 = nn.LayerNorm(num_embeddings)

  def forward(self, x):
    x = x + self.multihead(self.layernorm1(x))
    x = x + self.feedforward(self.layernorm2(x))
    return x

In [94]:
class MyLLM(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(VOCAB_SIZE, num_embeddings)
    self.position_embedding_table = nn.Embedding(block_size, num_embeddings)
    self.blocks = nn.Sequential(*[TransformerBlock(num_embeddings, num_heads) for _ in range(num_layers)])
    self.layernorm = nn.LayerNorm(num_embeddings)
    self.modeling_head = nn.Linear(num_embeddings, VOCAB_SIZE)

  def forward(self,idx, targets = None):
    token_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(idx.shape[1], device = device))
    x = token_emb + pos_emb
    x = self.blocks(x)
    x = self.layernorm(x)
    logits = self.modeling_head(x)
    if targets is None:
      loss = None
    else:
      num_batches, time_steps, channels = logits.shape
      logits = logits.view(num_batches*time_steps, channels)
      targets = targets.view(num_batches*time_steps)
      loss = F.cross_entropy(logits, targets)
    return logits, loss
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:] # these are the idx on which the generate will depend upon
      logits, loss = self(idx_cond) # these are the predictions
      logits = logits[:, -1, :] # only last prediction matters
      probs = F.softmax(logits, dim = -1) # get the probabilities
      idx_next = torch.multinomial(probs, num_samples = 1) # sample from the distribution so the token generated can be other than the one with maximum probability
      idx = torch.cat((idx, idx_next), dim = 1) # append the new index to the running sequence
    return idx


In [95]:
model = MyLLM()
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(), lr = learning_rate)

# I will also add a decaying learning rate when the loss function plateaus 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode = 'min',
    factor = 0.1,
    patience = 5,
    threshold = 1e-3,
    threshold_mode = 'rel'
    
)

print(f"Initial learning rate: {learning_rate}")

for iter in range(num_max_iterations):
  if iter % periodicity_of_evaluation == 0 or iter == num_max_iterations-1:
    losses = approximate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    scheduler.step(losses['val'])

    current_lr = optimizer.param_groups[0]['lr']
    print(f"Current learning rate: {current_lr}")
    
  X,Y = get_batch('train')

  logits, loss = model(X,Y)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()

context = torch.zeros((1,1),dtype = torch.long, device = device)
print(decode(m.generate(context, max_new_tokens = 500)[0].tolist(), loaded_vocab))

Initial learning rate: 0.001
step 0: train loss 9.1904, val loss 9.1898
Current learning rate: 0.001
step 100: train loss 6.2311, val loss 6.2404
Current learning rate: 0.001
step 200: train loss 5.7415, val loss 5.7800
Current learning rate: 0.001
step 300: train loss 5.4799, val loss 5.5366
Current learning rate: 0.001
step 400: train loss 5.2291, val loss 5.3147
Current learning rate: 0.001
step 500: train loss 5.0509, val loss 5.1566
Current learning rate: 0.001
step 600: train loss 4.9227, val loss 5.0504
Current learning rate: 0.001
step 700: train loss 4.8033, val loss 4.9504
Current learning rate: 0.001
step 800: train loss 4.7232, val loss 4.8779
Current learning rate: 0.001
step 900: train loss 4.6486, val loss 4.8258
Current learning rate: 0.001
step 1000: train loss 4.5855, val loss 4.7890
Current learning rate: 0.001
step 1100: train loss 4.5242, val loss 4.7351
Current learning rate: 0.001
step 1200: train loss 4.4770, val loss 4.7153
Current learning rate: 0.001
step 130

In [97]:

print("Training finished. Saving model...")
torch.save(model.state_dict(), 'model_weights2.pth')
print("Model saved!")

Training finished. Saving model...
Model saved!
